In [2]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct
import pickle

class CustomJPEGEncoder:
    def __init__(self, quality=50):
        self.quality = quality
        # Standard quantization matrices for luminance and chrominance
        self._lumQuant = np.array([
            [16, 11, 10, 16, 24, 40, 51, 61],
            [12, 12, 14, 19, 26, 58, 60, 55],
            [14, 13, 16, 24, 40, 57, 69, 56],
            [14, 17, 22, 29, 51, 87, 80, 62],
            [18, 22, 37, 56, 68, 109, 103, 77],
            [24, 35, 55, 64, 81, 104, 113, 92],
            [49, 64, 78, 87, 103, 121, 120, 101],
            [72, 92, 95, 98, 112, 100, 103, 99]
        ])
        self._chrQuant = np.array([
            [17, 18, 24, 47, 99, 99, 99, 99],
            [18, 21, 26, 66, 99, 99, 99, 99],
            [24, 26, 56, 99, 99, 99, 99, 99],
            [47, 66, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99],
            [99, 99, 99, 99, 99, 99, 99, 99]
        ])
        self._Qlum = self._scaleMatrix(self._lumQuant, quality)
        self._Qchr = self._scaleMatrix(self._chrQuant, quality)
        self._zigzagMap = self._computeZigzag(8)

    def _scaleMatrix(self, mat, q):
        if q < 50:
            factor = 5000 / q
        else:
            factor = 200 - 2 * q
        scaled = np.floor((mat * factor + 50) / 100)
        scaled[scaled == 0] = 1
        return scaled

    def _computeZigzag(self, n):
        indices = np.empty((n*n, 2), dtype=int)
        idx = 0
        for s in range(0, 2 * n - 1):
            if s % 2 == 0:
                x = min(s, n - 1)
                y = s - x
                while x >= 0 and y < n:
                    indices[idx] = [x, y]
                    idx += 1
                    x -= 1
                    y += 1
            else:
                y = min(s, n - 1)
                x = s - y
                while y >= 0 and x < n:
                    indices[idx] = [x, y]
                    idx += 1
                    x += 1
                    y -= 1
        return indices

    def _rgb2ycbcr(self, img):
        im = img.astype(np.float32)
        Y  = 0.299 * im[:, :, 0] + 0.587 * im[:, :, 1] + 0.114 * im[:, :, 2]
        Cb = -0.168736 * im[:, :, 0] - 0.331264 * im[:, :, 1] + 0.5 * im[:, :, 2] + 128
        Cr = 0.5 * im[:, :, 0] - 0.418688 * im[:, :, 1] - 0.081312 * im[:, :, 2] + 128
        return Y, Cb, Cr

    def _downsample(self, channel):
        return channel[::2, ::2]

    def _splitIntoBlocks(self, channel, blockSize=8):
        h, w = channel.shape
        pad_h = (blockSize - h % blockSize) % blockSize
        pad_w = (blockSize - w % blockSize) % blockSize
        padded = np.pad(channel, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
        blocks = []
        for i in range(0, padded.shape[0], blockSize):
            for j in range(0, padded.shape[1], blockSize):
                blocks.append(padded[i:i+blockSize, j:j+blockSize])
        return blocks, padded.shape

    def _applyDCT(self, block):
        shifted = block - 128
        return dct(dct(shifted.T, norm='ortho').T, norm='ortho')

    def _doQuantization(self, dctBlock, quantMat):
        return np.round(dctBlock / quantMat).astype(np.int32)

    def _zigzagScan(self, block):
        return np.array([block[i, j] for i, j in self._zigzagMap])

    def _runLengthEncode(self, array):
        encoded = []
        count = 0
        for num in array:
            if num == 0:
                count += 1
            else:
                encoded.append((count, num))
                count = 0
        encoded.append((0, 0))  # End marker
        return encoded

    def compress(self, imagePath, outFile="compressed.bin"):
        img = Image.open(imagePath).convert("RGB")
        imArr = np.array(img)
        Y, Cb, Cr = self._rgb2ycbcr(imArr)
        Cb = self._downsample(Cb)
        Cr = self._downsample(Cr)
        
        # Break each channel into blocks
        YBlocks, YShape = self._splitIntoBlocks(Y)
        CbBlocks, CbShape = self._splitIntoBlocks(Cb)
        CrBlocks, CrShape = self._splitIntoBlocks(Cr)
        
        # Process blocks for each channel
        encodeChannel = lambda blocks, Q: [
            self._runLengthEncode(self._zigzagScan(self._doQuantization(self._applyDCT(b), Q)))
            for b in blocks
        ]
        encodedY = encodeChannel(YBlocks, self._Qlum)
        encodedCb = encodeChannel(CbBlocks, self._Qchr)
        encodedCr = encodeChannel(CrBlocks, self._Qchr)
        
        payload = {
            'Y': encodedY,
            'Cb': encodedCb,
            'Cr': encodedCr,
            'Y_shape': YShape,
            'Cb_shape': CbShape,
            'Cr_shape': CrShape,
            'orig_shape': imArr.shape,
            'quality': self.quality
        }
        with open(outFile, "wb") as fp:
            pickle.dump(payload, fp)
        print("Compression complete. Data written to", outFile)

def main():
    encoder = CustomJPEGEncoder(quality=50)
    encoder.compress("Sample1.png", "compressed.bin") #TODO Change file name

if __name__ == "__main__":
    main()


Compression complete. Data written to compressed.bin
